# Import related functions

In [1]:
import sys
import cobra
import pandas as pd
import re
sys.path.append(r'./script/')#问题1
from ECMpy_function import *
from AutoPACMEN_function import *

# Input and output files

In [2]:
# Initial parameter
obj='Biomass
fluxes_outfile = './analysis/ECMpy_solution_%s_pfba.csv'%obj #？
use_substrate='R2399'# substrate ID
concentration=10 
# The total protein fraction in cell.

ptot = 0.4653
# The approximated average saturation of enzyme.

sigma = 0.5
# Lowerbound  of enzyme concentration constraint. 

lowerbound = 0   
enz_ratio=0.1

# Input files
autopacmen_folder = "./iDL1450_get_data/"
gap_fill= 'mean'#'mean'#'median'
project_name = "iDL1450%s"%gap_fill
organism = "Thermothelomyces thermophilus"
create_file(autopacmen_folder)
reaction_gene_path = "./data/reaction_gene_subunit_num.json"

protein_kcat_database_path = "none"
bigg_metabolites_file = "./data/bigg_models_metabolites.txt"#date:20221008 http://bigg.ucsd.edu/static/namespace/bigg_models_metabolites.txt
brenda_textfile_path = "./data/brenda_download.txt"#date:20221008 https://www.brenda-enzymes.org/brenda_download/file_download.php

sbml_path="./model/iYT1475.xml"

c13reaction_file='./reaction_kcat/C13data.csv'#C13 data


EC_max_file='./data/EC_kcat_max.json'
brenda_json_path = "%skcat_database_brenda.json"%autopacmen_folder
brenda_output_json_path = "%skcat_database_brenda_for_model.json"%autopacmen_folder
bigg_id_name_mapping_path = "%sbigg_id_name_mapping.json"%autopacmen_folder
sabio_rk_output_path = "%skcat_database_sabio_rk.json"%autopacmen_folder
combined_output_path = "%skcat_database_combined.json"%autopacmen_folder
reaction_kcat_MW_file="%s/reaction_kcat_MW.csv"%autopacmen_folder
uniprot_data_file='%s/%s_protein_id_mass_mapping.json'%(autopacmen_folder,project_name)


modelgene2uniprot_file='./analysis/modelgene2uniprot.txt'
paxdbgene2uniprot_file='./analysis/paxdbgene2uniprot.txt'
ecModel_output_file="./model/ecMTM_DLkcat.json" # 

json_output_file = './model/ecMTM_DLKcat_adj_enz.json'
reaction_kcat_MW_outfile="./get_reaction_kcat/kcat/reaction_change_by_enzuse_DLKcat.csv"
reaction_kcat_MW_round2_outfile='./get_reaction_kcat/kcat/reaction_change_by_C13_DLkcat.csv'


path exists


### Calculate f-value

In [3]:
model=cobra.io.read_sbml_model(sbml_path)
gene_abundance_file1 = './data/danbaizu/wt1.csv'
gene_abundance_file2 = './data/danbaizu/wt2.csv'
gene_abundance_file3 = './data/danbaizu/wt3.csv'
gene_abundance_file4 = './data/danbaizu/wt4.csv'
file_list=[gene_abundance_file1,gene_abundance_file2,gene_abundance_file3,gene_abundance_file4]
f_list=[]
for g_a_file in file_list:
    f=calculate_f_special_new(model,g_a_file)
    f_list.append(f)
f_list

[0.5563635421646973, 0.5540772163224917, 0.558660486869679, 0.5486668910172237]

# Constructing Enzyme-Constrained Mode

In [5]:
#The enzyme mass fraction 
f = round(np.mean(f_list), 2)
# print("f is:" + str(f))
ptot = 0.4653
# upperbound =0.128
upperbound = round(ptot * f * sigma, 3)
# upperbound = 0.1
print("upperbound is:" + str(upperbound))
reaction_kcat_MW_file="./get_reaction_kcat/kcat/DLKcat3.30.csv"# kcat data predicted by DLKcat method

trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, ecModel_output_file)

enz_model=get_enzyme_constraint_model(ecModel_output_file)
enz_model.objective=obj
enz_model.summary()
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-10, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_output_file)
print(enz_model_pfba_solution.fluxes[obj])

upperbound is:0.128
0.128
IN FLUXES           OUT FLUXES    OBJECTIVES
------------------  ------------  ---------------
o2_e      1.47      h2o_e  1.91   Biomass  0.0219
h_e       1.21      co2_e  1.5
glc__D_e  0.452     ac_e   0.161
nh4_e     0.164
pi_e      0.00397
so4_e     0.000317
Enzyme cost total is:0.128
0.021879981734168438


# Calibration kcat according Enzyme usage

In [6]:
#Initial model and result
fluxes_infile_ori = './analysis/ECMpy_solution_%s_pfba.csv'%obj
fluxes_outfile = fluxes_infile_ori
need_change_reaction_list=[]
changed_reaction_list=[]
round_num=1


reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#Model adjustment
enz_model_obj_max=0
concentration = 3.0676
# Reconstruction and analysis of genome-scale metabolic model for thermophilic fungus Myceliophthora thermophila [J]. Biotechnology and Bioengineering, 2022, 119(7): 1926-37.
# Beyond growth rate 0.25: Elucidation of the metabolic mechanism for malate production in Myceliophthora thermophila via 13C metabolic flux analysis [Z]. Research Square. 2022.10.21203/rs.3.rs-2123109/v1
while (enz_model_obj_max<0.2573):

 
    
    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(enz_ratio,\
                                                    ecModel_output_file,fluxes_outfile,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
    
    #Using only the provided substrate
   
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
    enz_model.reactions.get_by_id(use_substrate).bounds = (-concentration, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)
  
  
    
    enz_model_obj=enz_model.slim_optimize()

    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    fluxes_outfile = './analysis/ECMpy_solution_adjround%s.csv'%(round_num)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_output_file)
    
    

   # μmax condition substrate concentration
 
    obj='Biomass'
    substrate='R2399'
    enz_model.reactions.get_by_id(substrate).bounds=(-1000,0)
    enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,0)
    
    enz_model.objective=obj
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    print("ecGEM max glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
    print("ecGEM max growth rates:"+str(enz_model_solution.fluxes[obj]))
    enz_model_sub=-enz_model_solution.fluxes[substrate]
    enz_model_obj_max=enz_model_solution.fluxes[obj]


    print('Round %s: '%round_num+str(enz_model_obj))
    round_num=round_num+1
    if round_num>50:
        break
    print()
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)

Need changing reaction: 
R2160
Changed reaction: 
['R2160']
0.128
Enzyme cost total is:0.128
Enzyme cost total is:0.12800000000000003
ecGEM max glucose uptake rates:0.8418663312187247
ecGEM max growth rates:0.060579546752058046
Round 1: 0.06057954675205813

Need changing reaction: 
R2158
Changed reaction: 
['R2160']
0.128
Enzyme cost total is:0.12800000000000003
Enzyme cost total is:0.12800000000000003
ecGEM max glucose uptake rates:0.8418663312187247
ecGEM max growth rates:0.060579546752058046
Round 2: 0.06057954675205813

Need changing reaction: 
R2165
Changed reaction: 
['R2160']
0.128
Enzyme cost total is:0.12800000000000003
Enzyme cost total is:0.12800000000000003
ecGEM max glucose uptake rates:0.8418663312187247
ecGEM max growth rates:0.060579546752058046
Round 3: 0.06057954675205813

Need changing reaction: 
R2306_reverse_num2
Changed reaction: 
['R2160']
0.128
Enzyme cost total is:0.12800000000000003
Enzyme cost total is:0.12800000000000003
ecGEM max glucose uptake rates:0.8418

Changed reaction: 
['R2160', 'R25', 'R61', 'R16', 'R255_num2', 'R351', 'R145', 'R56_num3', 'R14', 'R1442_num2', 'R72', 'R1378_num2', 'R50', 'R70', 'R66_num5', 'R1122_num1', 'R914', 'R1377']
0.128
Enzyme cost total is:0.128
Enzyme cost total is:0.128
ecGEM max glucose uptake rates:1.3326354931700144
ecGEM max growth rates:0.09231442058047024
Round 26: 0.09231442058047105

Need changing reaction: 
R1744
Changed reaction: 
['R2160', 'R25', 'R61', 'R16', 'R255_num2', 'R351', 'R145', 'R56_num3', 'R14', 'R1442_num2', 'R72', 'R1378_num2', 'R50', 'R70', 'R66_num5', 'R1122_num1', 'R914', 'R1377', 'R1744']
0.128
Enzyme cost total is:0.128
Enzyme cost total is:0.12800000000000006
ecGEM max glucose uptake rates:1.34331290051594
ecGEM max growth rates:0.09321836163464835
Round 27: 0.09321836163464854

Need changing reaction: 
R244
Changed reaction: 
['R2160', 'R25', 'R61', 'R16', 'R255_num2', 'R351', 'R145', 'R56_num3', 'R14', 'R1442_num2', 'R72', 'R1378_num2', 'R50', 'R70', 'R66_num5', 'R1122_num1

0.128
Enzyme cost total is:0.1280000000000001
Enzyme cost total is:0.1280000000000001
ecGEM max glucose uptake rates:1.4377732955061477
ecGEM max growth rates:0.10083303546792943
Round 42: 0.10083303546792928

Need changing reaction: 
R9
Changed reaction: 
['R2160', 'R25', 'R61', 'R16', 'R255_num2', 'R351', 'R145', 'R56_num3', 'R14', 'R1442_num2', 'R72', 'R1378_num2', 'R50', 'R70', 'R66_num5', 'R1122_num1', 'R914', 'R1377', 'R1744', 'R244', 'R1455_num1', 'R254_num1', 'R185_num1', 'R936', 'R1822', 'R1836', 'R74', 'R54', 'R42_num1', 'R41_num1', 'R51', 'R9']
0.128
Enzyme cost total is:0.1280000000000001
Enzyme cost total is:0.12800000000000003
ecGEM max glucose uptake rates:1.3581819366730201
ecGEM max growth rates:0.10203552572417553
Round 43: 0.10203552572417572

Need changing reaction: 
R1583
Changed reaction: 
['R2160', 'R25', 'R61', 'R16', 'R255_num2', 'R351', 'R145', 'R56_num3', 'R14', 'R1442_num2', 'R72', 'R1378_num2', 'R50', 'R70', 'R66_num5', 'R1122_num1', 'R914', 'R1377', 'R1744

In [8]:

c13_percentage=0.1
# kcat_database_combined_file='./data/EC_kcat_max.json'##max kcat for EC number selected from BRENDA and SABIO-RK database(use autoPACMEN)
model_name=sbml_path
f=0.55 # Enzyme mass fraction
ptot=0.4653 # total protein content in cells
sigma=0.5 #approximate average saturation of the enzyme
lowerbound= 0 
upperbound=round(ptot * f * sigma, 3)

json_round2_output_file='./model/eciYT1475_DL.json'


enz_model=get_enz_model_use_c13(reaction_kcat_MW_outfile, ecModel_output_file, c13reaction_file, c13_percentage, \
                                EC_max_file,model_name, f, ptot, sigma, lowerbound, \
                                upperbound, json_round2_output_file,reaction_kcat_MW_round2_outfile)
enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,3.06)
enz_model.summary()

0.128
通量为： 0.0
R6_num1 ECMpy_c13_reaction_flux: 0.00028451498566400005 flux: 3.0676
0.128
通量为： 0.0
R7 ECMpy_c13_reaction_flux: 0.00295410374016 flux: 1.4289
0.128
通量为： 0.0
R1036 ECMpy_c13_reaction_flux: 3.15923569216e-05 flux: 1.936
0.128
通量为： 0.0
R1044_num1 ECMpy_c13_reaction_flux: 0.002698297463040001 flux: 1.936
0.128
通量为： 0.39378958331877567
R50 ECMpy_c13_reaction_flux: 0.158462757914232 flux: 4.1708
0.128
通量为： 0.39378958331877567
R51 ECMpy_c13_reaction_flux: 1.3639262432400694 flux: 4.1708
0.128
通量为： 0.0
R42_num2 ECMpy_c13_reaction_flux: 0.006508857785600001 flux: 3.9965
0.128
通量为： 0.3793340141337602
R40 ECMpy_c13_reaction_flux: 0.022370116505600002 flux: 3.9965
0.128
通量为： 0.19880746452182474
R36_num2 ECMpy_c13_reaction_flux: 4.854059742128076 flux: 3.8319
0.128
通量为： 0.0
R55 ECMpy_c13_reaction_flux: 0.0031095719616000004 flux: 1.2271
0.128
通量为： 0.7483709222441511
R60 ECMpy_c13_reaction_flux: 0.006412713715200001 flux: 1.2271
0.128
通量为： 1.195172944208321
R56_num3 ECMpy_c13_reaction

# Obtain the metabolic flux of this model

In [3]:

ecModel_output_file='./model/eciYT1475_DL.json'

enz_model=get_enzyme_constraint_model(ecModel_output_file)

enz_model.reactions.get_by_id('R2399_reverse').bounds=(0,3.06)
enz_model.reactions.get_by_id("R1975_num2").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num3").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num4").bounds=(0,0)
enz_model.reactions.get_by_id("R1975_num5").bounds=(0,0)


solution = enz_model.optimize()
reac=dict(solution.fluxes[solution.fluxes> 0.1])

genelist=[]
reactionlist=[]
fluxlist=[]
for i in reac:
    #
    reID=str(enz_model.reactions.get_by_id(i))
    reID=str(enz_model.reactions.get_by_id(i)).split(':')[0]
   
    reID= reID.split('_')[0]
    reactionlist.append(reID)
    fluxlist.append(reac[i])
df=pd.DataFrame({'reaction':reactionlist,'flux':fluxlist})

df.to_csv("./flux/flux for DL.csv",index=0)
# enz_model.summary()

0.128
